# In this notebook, we're going to cover some of the most fundamental 

More specifically, we're going to cover:
- Introduction to tensors
- Getting information from tensors
- Manipulating tensors
- Tensors & NumPy
- Using @tf.function(a way to speed up your regular Python functions)
- Using GPUs with TensorFlow (or TPU's)
- Excercises to try for yourself

## Introduction to Tensors

In [1]:
# Import TensorFlow
import tensorflow as tf 
print(tf.__version__)

2.9.2


In [2]:
# Create tensors with tf.constant()
scaler = tf.constant()
scaler

Metal device set to: Apple M1


2022-06-15 00:32:27.049050: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-15 00:32:27.050070: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [4]:
# Check the number of dimensions of a tensor (ndim stands for number of dimensions)
scaler.ndim

0

In [5]:
#Create a vector 
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [6]:
# Check the number of dimensions of our vector
vector.ndim

1

In [7]:
# Create a matrix (has more than 1 dimension)
matrix = tf.constant(([10,7],
                     [7,10]))
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [8]:
matrix.ndim

2

In [10]:
#Create another matrix
another_matrix = tf.constant([[10.,7.],
                             [7.,10.],
                              [8.,9.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 7., 10.],
       [ 8.,  9.]], dtype=float16)>

In [11]:
another_matrix.ndim

2

In [19]:
#Creating a tensor
tensor = tf.constant([[[1,2,3],
                     [4,5,6]],
                     [[7,8,9],
                     [10,11,12]],
                    [[13,14,15],
                    [16,17,18]]])

tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [20]:
tensor.ndim

3

What we've created so far:

- Scaler: a single number
- Vector: a number with direction (e.g. wind speed and direction)
- Matrix: a 2-dimensional array of numbers
- Tensor: an n-dimensional array of numbers (when n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with `tf.Variable()`

In [21]:
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [22]:
changeable_tensor[0] = [1]

TypeError: 'ResourceVariable' object does not support item assignment

In [23]:
changeable_tensor[0].assign(1)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([1, 7], dtype=int32)>

**Note**: Rarely in practice will you need to decide whether to use `tf.constant` or `tf.Variable` to create tensors, as TensorFlow does this for you. However, if in doubt, use `tf.constant` and change it later

### Creating random tensors

Random tensors are tensors of some arbitrary size when contains random numbers.

In [28]:
#create two random (but the same) tensors
with tf.device('/cpu:0'): #for m1 macs (not needed otherwise)
    random_1 = tf.random.Generator.from_seed(42)
    random_1 = random_1.normal(shape=(3,2))
    random_2 = tf.random.Generator.from_seed(42)
    random_2 = random_2.normal(shape=(3,2))
    
# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements of tensors

In [29]:
# Shuffle a tensor (shuffle split)
not_shuffled = tf.constant(([10,7],
                     [7,10]))

#Shuffle our non-shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 7, 10],
       [10,  7]], dtype=int32)>

In [30]:
tf.ones([3,2])

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>

In [31]:
tf.zeros(shape=(3,2))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [32]:
#Turning numpy arrays to tensors 
import numpy as np

numpy_A = np.arange(1,25, dtype=np.int32) 
numpy_A 
#X = tf.constant(some_matrix) # Captial for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [36]:
A = tf.constant(numpy_A)
B = tf.constant(numpy_A, shape=(2,3,4))

A,B

(<tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>,
 <tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>)

In [35]:
A = tf.constant(numpy_A, shape=(2,3,4))
A

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

### Getting information from tensors

When dealing with tensors you probably want to be aware of the following attributes:
- Shape
- Rank
- Axis or dimension
- Size

In [37]:
#Creating a rank 4 tensor (4-dimensions)
rank_4_tensor = tf.zeros(shape= [2,3,4,5])
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [38]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [44]:
print(f' the tensor shape is {rank_4_tensor.shape},\n the rank is {rank_4_tensor.ndim},\n and size is {tf.size(rank_4_tensor)}')

 the tensor shape is (2, 3, 4, 5),
 the rank is 4,
 and size is 120


In [45]:
2*3*4*5

120

In [49]:
#Get various attributes of our tensor
print('Datatype of every element:', rank_4_tensor.dtype)
print('Number of dimensions (rank):', rank_4_tensor.ndim)
print('Shape of tensor:', rank_4_tensor.shape)
print('Elements along the 0 axis:', rank_4_tensor.shape[0])
print('Elements along the last axis:', rank_4_tensor.shape[-1])
print('Total number of elements in our tensor:', tf.size(rank_4_tensor))
print('Total number of elements in our tensor:', tf.size(rank_4_tensor).numpy())

Datatype of every element: <dtype: 'float32'>
Number of dimensions (rank): 4
Shape of tensor: (2, 3, 4, 5)
Elements along the 0 axis: 2
Elements along the last axis: 5
Total number of elements in our tensor: tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor: 120


### Indexing tensors 

Tensors can be indexed like python lists

In [50]:
 rank_4_tensor[:2,:2,:2,:2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [51]:
#Get the first element from each dimension from each index except for the final one 
rank_4_tensor[:1,:1,:1,:]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [57]:
#Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[1,2],
                            [3,4]])
rank_2_tensor, rank_2_tensor.ndim

(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4]], dtype=int32)>,
 2)

In [58]:
# get the last item of each row of our rank 2 tensor
rank_2_tensor[:,-1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([2, 4], dtype=int32)>

In [59]:
# add extra dimension to our rank 2 tensor

rank_3_tensor = rank_2_tensor[...,tf.newaxis] #alternative of [:,:,tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

In [61]:
#alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis = -1) # axis = -1 means expand on the final axis
# axis = 0 means expand on the beginning axis
# axis = 1 means expand on the middle axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[1],
        [2]],

       [[3],
        [4]]], dtype=int32)>

### Manipulating Tensors (Tensor operations)

**Basic Operations**

`+`, `-`, `*`, `/`

In [66]:
tensor = tf.constant([[10,7],[3,4]])
tensor + 10


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [65]:
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [68]:
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

**Matrix Multiplication** 

In machine learning, matrix manipulation is one of the most common tensor operations.

There are two rules our tensors (or matrices) need to fulfil if we're going to matrix multiply them: 

1. The inner dimensions must match (e.g. 3x2 & 2x3; cannot with 3x2 & 3x2)
2. The resulting matrix has the shape of the outer dimensions

In [70]:
# Matrix multiplication in Tensorflow
print(tensor)
tf.matmul(tensor,tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [71]:
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [72]:
#Matrix multiplcation with python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [75]:
 #Create a tensor (3,2)
X = tf.constant([[1,2],
                [3,4],
                [5,6]])
# Create a tensor (3,2)
y = tf.constant([[7,8],
                [9,10],
                [11,12]])
X, y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [76]:
# lets try to matrix multiply 
X @ y

InvalidArgumentError: Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul]

In [77]:
# try using tf.matmul

tf.matmul(X,y)

InvalidArgumentError: Matrix size-incompatible: In[0]: [3,2], In[1]: [3,2] [Op:MatMul]

In [78]:
tf.transpose(y, perm=[3,2])

InvalidArgumentError: 3 is out of range [0 .. 2) [Op:Transpose]

In [79]:
# lets change the shape of y
tf.reshape(y, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [80]:
# try to multiply with reshaped y
X @ tf.reshape(y, shape=(2,3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [81]:
 tf.matmul(X, tf.reshape(y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

Resulting shape is (3,3) - fulfilling rule 1

Now lets try to reshape X instead

In [82]:
tf.reshape(X, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [83]:
tf.reshape(X, shape=(2,3)) @ y

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [84]:
tf.reshape(X, shape=(2,3)).shape, y.shape

(TensorShape([2, 3]), TensorShape([3, 2]))

Since the outer dimensions are 2x2 hence the shape of our matrix is 2x2 - Rule 2

In [88]:
 # can do the same with tf.transpose
# however, tf.transpose flips the axis
tf.transpose(X), tf.reshape(X, shape=(2,3)), X

(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 5],
        [2, 4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>)

**The Dot Product**

Matrix multiplication is also referred to as the dot product.

You can perform matrix mulitplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`

In [90]:
#Perform the dot product on X and y (requires X or y to be transposed)
tf.tensordot(tf.transpose(X), y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [91]:
 #Perform matrix multiplication between X and y (transposed)
tf.matmul(X, tf.transpose(y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [93]:
#Perform matrix multiplication between X and y (reshaped)
tf.matmul(X, tf.reshape(y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

Generally, when performing matrix multiplication on two tensors and one of the axes doesn't line up, you will transpose (rather than reshape) one of the tensors to get satisfy the matrix multiplication

### Changing the datatype of a tensor

In [97]:
#Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B, B.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 tf.float32)

In [98]:
C = tf.constant([7,10])
C.dtype

tf.int32

In [99]:
# change from float 32 to float 16 bit (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>

In [101]:
# change from int32 to float32
E = tf.cast(C, dtype = tf.float32)
E, E.dtype

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>,
 tf.float32)

###  Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values

In [102]:
# Get the absolute values 
F = tf.constant([-7,-10])
F

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [103]:
tf.abs(F)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregation:
- get the minimum
- get the max
- get the mean
- get the sum

In [104]:
G = tf.constant(np.random.randint(0,100, size = 50))
G

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([63, 53, 52,  0, 89, 37, 96, 47, 15, 67, 96, 53, 29, 98, 34, 32, 54,
       34, 81, 90, 90,  2, 92,  3, 78, 62, 87, 96, 71, 75, 85, 79, 46, 97,
       73, 27, 74,  9, 36, 47, 25, 88,  0, 39, 52, 21,  3, 44, 21, 73])>

In [105]:
tf.size(G), G.shape, G.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [106]:
tf.reduce_min(G)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [107]:
tf.reduce_max(G)

<tf.Tensor: shape=(), dtype=int64, numpy=98>

In [108]:
tf.reduce_mean(G)

<tf.Tensor: shape=(), dtype=int64, numpy=54>

In [109]:
tf.reduce_sum(G)

<tf.Tensor: shape=(), dtype=int64, numpy=2715>

In [118]:
tf.math.reduce_variance(tf.cast(G, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=913.73>

In [119]:
tf.math.reduce_std(tf.cast(G, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=30.227968>

In [114]:
import tensorflow_probability as tfp

In [115]:
tfp.stats.variance(G)

<tf.Tensor: shape=(), dtype=int64, numpy=913>

In [116]:
tfp.stats.stddev(G)

InvalidArgumentError: Value for attr 'T' of int64 is not in the list of allowed values: bfloat16, half, float, double, complex64, complex128
	; NodeDef: {{node Sqrt}}; Op<name=Sqrt; signature=x:T -> y:T; attr=T:type,allowed=[DT_BFLOAT16, DT_HALF, DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_COMPLEX128]> [Op:Sqrt]

### Find the positional maximum and minimum

In [120]:
# Create a new tensor for finding the positional minimum and maximum
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [121]:
#Find the positional maximum
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [122]:
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [123]:
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [124]:
# check if equal
F[tf.argmax(F)] == tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=bool, numpy=True>

###  Squeezing a tensor

In [125]:
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.68789124, 0.48447883, 0.9309944 , 0.252187  , 0.73115396,
           0.89256823, 0.94674826, 0.7493341 , 0.34925628, 0.54718256,
           0.26160395, 0.69734323, 0.11962581, 0.53484344, 0.7148968 ,
           0.87501776, 0.33967495, 0.17377627, 0.4418521 , 0.9008261 ,
           0.13803864, 0.12217975, 0.5754491 , 0.9417181 , 0.9186585 ,
           0.59708476, 0.6109482 , 0.82086265, 0.83269787, 0.8915849 ,
           0.01377225, 0.49807465, 0.57503664, 0.6856195 , 0.75972784,
           0.908944  , 0.40900218, 0.8765154 , 0.53890026, 0.42733097,
           0.401173  , 0.66623247, 0.16348064, 0.18220246, 0.97040176,
           0.06139731, 0.53034747, 0.9869994 , 0.4746945 , 0.8646754 ]]]]],
      dtype=float32)>

In [126]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [127]:
G_squeezed = tf.squeeze(G)

In [128]:
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.68789124, 0.48447883, 0.9309944 , 0.252187  , 0.73115396,
        0.89256823, 0.94674826, 0.7493341 , 0.34925628, 0.54718256,
        0.26160395, 0.69734323, 0.11962581, 0.53484344, 0.7148968 ,
        0.87501776, 0.33967495, 0.17377627, 0.4418521 , 0.9008261 ,
        0.13803864, 0.12217975, 0.5754491 , 0.9417181 , 0.9186585 ,
        0.59708476, 0.6109482 , 0.82086265, 0.83269787, 0.8915849 ,
        0.01377225, 0.49807465, 0.57503664, 0.6856195 , 0.75972784,
        0.908944  , 0.40900218, 0.8765154 , 0.53890026, 0.42733097,
        0.401173  , 0.66623247, 0.16348064, 0.18220246, 0.97040176,
        0.06139731, 0.53034747, 0.9869994 , 0.4746945 , 0.8646754 ],
       dtype=float32)>,
 TensorShape([50]))

### One-hot endcoding tensors

In [130]:
#Create a list of indices 
some_list = [0,1,2,3] #Could be red, green, blue, purple

#One hot encode our list of indices 
tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [131]:
# Specify customer values for one hot encoding 
tf.one_hot(some_list, depth=4, on_value="Yo! i love deep learning", \
           off_value="I like to dance")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'Yo! i love deep learning', b'I like to dance',
        b'I like to dance', b'I like to dance'],
       [b'I like to dance', b'Yo! i love deep learning',
        b'I like to dance', b'I like to dance'],
       [b'I like to dance', b'I like to dance',
        b'Yo! i love deep learning', b'I like to dance'],
       [b'I like to dance', b'I like to dance', b'I like to dance',
        b'Yo! i love deep learning']], dtype=object)>

### Squaring, log, square root

In [132]:
# Create a new tensor
H = tf.range(1,10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [133]:
#Square 
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [136]:
#Square root
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [137]:
# log
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986122, 1.3862944, 1.6094378, 1.7917594,
       1.94591  , 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy

TensorFlow interacts beautifully with NumPy arrays

**NOTE:** One of the main differences between TF and NP are is that TF can be run on a GPU or TPU for a faster processing 

In [138]:
# Create a tensor directly from a numpy array
J = tf.constant(np.array([3.,7.,10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [139]:
 #Coverting our tensor back to numpy
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [140]:
#conert tensor J to a numpy array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [142]:
J = tf.constant([3.])
J.numpy()[0]

3.0

In [143]:
# the default type of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
#Check the datatype of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

###  Finding access to GPU's

In [145]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [146]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found
